# Final Project

## Introduction

In the following development we will tend to reproduce numeralically the results published by Sussillo and Abbott onto their papers: 'Generating Coherent Patterns of Activity from Chaotic Neural Networks'.
In order to stuck to easily reproducable and understandable results, our results we be based on a complete coding of the learning algorithm avoiding the use of any previously coded librairies. 



## Paper abstract reminder

Neural circuits display complex activity patterns both spontaneously and when responding to a stimulus or generating a motor output. How are these two forms of activity related? We develop a procedure called FORCE learning for modifying synaptic strengths either external to or within a model neural network to change chaotic spontaneous activity into a wide variety of desired activity patterns. FORCE learning works even though the networks we train are spontaneously chaotic and we leave feedback loops intact and unclamped during learning. Using this approach, we construct networks that produce a wide variety of complex output patterns, input-output transformations that require memory, multiple outputs that can be switched by control inputs, and motor patterns matching human motion capture data. Our results reproduce data on premovement activity in motor and premotor cortex, and suggest that synaptic plasticity may be a more rapid and powerful modulator of network activity than generally appreciated.


## Results - (Sussillo and Abbott figure 2 reproduction)

## Examples of FORCE Learning (Sussillo and Abbott figure 3 reproduction)